# 玩转 Metop AVHRR数据

### 摘要在本教程中，我们将读取Metop AVHRR数据，并显示几个合成图像，采用卫星投影并在几个区域上展示，同时向部分图像添加海岸线。

## 查找文件，按时间过滤要求：
- 保留 Markdown 格式
- 保留代码、URL 和 HTML 标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

要翻译的文本：
本教程的数据包含少量 Metop-B AVHRR/3 数据粒（granules），这些数据跨越数小时的时间。由于我们只需要处理这些数据的一部分，一个解决方案是定义开始和结束时间，并过滤提供给 `Scene` 的文件。

In [ ]:
from satpy import Scene, find_files_and_readers
from datetime import datetime
import pprint

files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex2',
                               reader='avhrr_l1b_eps',
                               start_time=datetime(2018, 10, 7 ,9, 25),
                               end_time=datetime(2018, 10, 7 , 9, 30))

#files=sorted(files)
scn = Scene(filenames=files)
pprint.pprint(files)

- 然后我们决定加载一个组合件并在屏幕上显示它

In [ ]:
print(scn.available_composite_names())

In [ ]:
composite = 'overview'
scn.load([composite])
scn.show(composite)


## 重采样与海岸线显示重采样与上一个教程中的操作完全相同。不过这一次，我们将在数据上显示海岸线。

In [ ]:
newscn = scn.resample('euro1')


In [ ]:
newscn.show(composite, overlay={'coast_dir': '/tcenas/scratch/pytroll/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})


要了解可用于海岸线燃烧的参数，请查阅pycoast文档。

## 动态区域重采样对于极地数据，可以方便地在给定投影下对数据进行重采样，但需要将区域范围调整为实际重采样的数据。为此，目前我们有两种可用的动态区域：`omerc_bb` 用于斜轴墨卡托投影，`laea_bb` 用于兰伯特等积方位角投影。_注意_: 此处 `laea_bb` 对于所使用的数据集不会产生有意义的结果。

In [ ]:
# Resample and show the image
newscn = scn.resample('omerc_bb')
newscn.show(composite, overlay={'coast_dir': '/tcenas/scratch/pytroll/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})


## 按区域过滤数据当拥有大量数据粒度时，仅加载覆盖特定区域的数据可能更方便。我们可以通过在 `Scene` 实例化期间使用参数过滤来实现这一点。

In [ ]:
files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex2',
                               reader='avhrr_l1b_eps')
scn = Scene(filenames=files, filter_parameters={'area':'germ'})

In [ ]:
composite = 'natural_color'
scn.load([composite])
newscn = scn.resample('eurol')
newscn.show(composite, overlay={'coast_dir': '/tcenas/scratch/pytroll/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})

## 保存数据Satpy 允许您以各种格式保存数据，例如 GeoTIFF

In [ ]:
newscn.save_dataset(composite,filename='my_fantastic_image.tiff')
#newscn.save_dataset(composite,filename='my_fantastic_image.nc')

### 练习 2.1使用单个通道生成一个 `ndvi` 产品，并在 `euron1` 区域上显示结果。然后将您的工作保存为PNG。
NDVI公式: ndvi = (vis08 - vis06) / (vis08 + vis06)